## Constructors

Objects of `struct` types can be initialized using the `{` `}` initializer syntax specifying the components. However, in some situations, we may want different non-trivial kinds of initialization. Any kind of initialization can be performed for a class with any number of desired inputs. The initialization functions are called "Constructors" and they have the same name as the class. Every time an object comes into existence, it is implicitly or explicitly a call to one of these initialization functions or constructors.

Similarly, every time an object runs out of scope and is to be removed from the program, a different "destructor" function is **automatically** called. Constructors are typically overloaded: there are many ways to create an object. The destructor for a class is unique.

### Demo with the Vbose class

In [1]:
#pragma cling add_include_path("/p/project/training2213/local/include")

In [2]:
#include <iostream>
#include <vector>
#include "Vbose.hh"

In your course material, you will find, in the folder called `code`, a header called `Vbose.hh`. `Vbose` is a class where the special members like constructors, destructors are written in such a way that they emit messages when they are used. Calls to copy/move constructors and destructors usually happen silently behind the scenes. They follow strict rules about scopes, but usually we don't get to see them in action explicitly. The verbose versions of those functions in `Vbose` makes them visible. For instance...

In [3]:
{
    std::cout << "About to create first Vbose object\n";
    Vbose one{"FIRST"};
    std::cout << "Now creating another Vbose object using copy construction syntax...\n";
    auto two{one};
    std::cout << "Creating an alias to the first object...\n";
    auto& three{one};
    auto four{one + two};
}

About to create first Vbose object
Constructor of object at 140640271797832, using string "FIRST"
Now creating another Vbose object using copy construction syntax...
Copy constructor of object at 140640271797744. Source for copy is at 140640271797832
Creating an alias to the first object...
Inside operator + ()
Constructor of object at 140640271797704, using string "FIRST+FIRST"
Destructor of object at 140640271797704 with data "FIRST+FIRST"
Destructor of object at 140640271797744 with data "FIRST"
Destructor of object at 140640271797832 with data "FIRST"


The construction of the variable `four` happens using what is known as Return Value Optimization (RVO) which obviates what would otherwise have been a move construction. However, there are other situations where move constructor is really automatically used...

In the following, we have a pre-existing object for `four` instead of creating it newly directly from the result of the `+` operation. The object is old, so it can not be newly constructed using RVO. It is an assignment operation, taking a temporary value as input. What version of the assignment operator do you expect will be used? Run the following cell to see if your expectation is borne out!

In [6]:
{
    std::cout << "About to create first Vbose object\n";
    Vbose one{"FIRST"};
    std::cout << "Now creating another Vbose object using copy construction syntax...\n";
    auto two{one};
    std::cout << "Creating an alias to the first object...\n";
    auto& three{one};
    Vbose four{"FOUR"};
    four = one + two;
}

About to create first Vbose object
Constructor of object at 140640271797832, using string "FIRST"
Now creating another Vbose object using copy construction syntax...
Copy constructor of object at 140640271797744. Source for copy is at 140640271797832
Creating an alias to the first object...
Constructor of object at 140640271797704, using string "FOUR"
Inside operator + ()
Constructor of object at 140640271797632, using string "FIRST+FIRST"
Move assignment operator: LHS @ 140640271797704("FOUR"), RHS @ 140640271797632("FIRST+FIRST")
Destructor of object at 140640271797632 with data "FOUR"
Destructor of object at 140640271797704 with data "FIRST+FIRST"
Destructor of object at 140640271797744 with data "FIRST"
Destructor of object at 140640271797832 with data "FIRST"


`push_back` operations for `std::vector` cause sporadic calls to lots of copy/move operations.

In [8]:
{
    std::vector<Vbose> vv;
    // vv.reserve(10); // what happens if you uncomment this line?
    Vbose one{"FIRST"};
    for (int i = 0; i < 10; ++i) vv.push_back(one);
}

Constructor of object at 140000934993456, using string "FIRST"
Copy constructor of object at 140000320763856. Source for copy is at 140000934993456
Copy constructor of object at 140000320443712. Source for copy is at 140000934993456
Move constructor of object at 140000320443680. Source for move is at 140000320763856
Destructor of object at 140000320763856 with data ""
Copy constructor of object at 140000320763616. Source for copy is at 140000934993456
Move constructor of object at 140000320763552. Source for move is at 140000320443680
Destructor of object at 140000320443680 with data ""
Move constructor of object at 140000320763584. Source for move is at 140000320443712
Destructor of object at 140000320443712 with data ""
Copy constructor of object at 140000320763648. Source for copy is at 140000934993456
Copy constructor of object at 140000320341056. Source for copy is at 140000934993456
Move constructor of object at 140000320340928. Source for move is at 140000320763552
Destructor of

Finally, we can use the Vbose class to see copy happening when we loop over a container using a plain `auto` as opposed to `auto&&` in a range based for loop...

In [6]:
{
    using namespace std::string_literals;
    std::vector<Vbose> vv(10, Vbose("EXEMPLAR"));
    for (auto elem: vv) std::cout << elem.value() << "\n";
}

Constructor of object at 140000934993456, using string "EXEMPLAR"
Copy constructor of object at 140000318512896. Source for copy is at 140000934993456
Copy constructor of object at 140000318512928. Source for copy is at 140000934993456
Copy constructor of object at 140000318512960. Source for copy is at 140000934993456
Copy constructor of object at 140000318512992. Source for copy is at 140000934993456
Copy constructor of object at 140000318513024. Source for copy is at 140000934993456
Copy constructor of object at 140000318513056. Source for copy is at 140000934993456
Copy constructor of object at 140000318513088. Source for copy is at 140000934993456
Copy constructor of object at 140000318513120. Source for copy is at 140000934993456
Copy constructor of object at 140000318513152. Source for copy is at 140000934993456
Copy constructor of object at 140000318513184. Source for copy is at 140000934993456
Destructor of object at 140000934993456 with data "EXEMPLAR"
Copy constructor of obj